In [1]:
from music21 import converter
from music21 import instrument
from music21 import chord
from music21 import note
import glob

def get_notes(glob_path):
    notes = []
    
    print(len(glob.glob(glob_path)))
    for input_file in glob.glob(glob_path):
        print(input_file)
        midi = converter.parse(input_file)

        parts = instrument.partitionByInstrument(midi)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes




notes = get_notes('bowser/*.mid')
print(notes)
            
# get amount of pitch names
n_vocab = len(set(notes))

6
bowser/fortress.mid
bowser/bowser.mid
bowser/castle.mid
bowser/boss-2-.mid
bowser/final-bowser-theme.mid
bowser/boss.mid
['B-2', 'C3', 'A3', 'B3', 'G#3', 'B-3', 'B-2', 'C3', 'A3', 'B3', 'G#3', 'B-3', 'G#2', 'B-2', 'G#3', 'G3', 'C3', 'B3', 'C#3', 'E3', 'G#2', 'A3', 'B-2', 'C3', 'A3', 'B3', 'G#3', 'B-3', 'B-2', 'C3', 'A3', 'B3', 'G#3', 'B-3', 'G#2', 'B-2', 'G#3', 'G3', 'C3', 'B3', 'C#3', 'E3', 'G#2', 'A3', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', 'D2', '6.11', 'C#4', 'F#2', 'C#4', '6.11', '10.2', 'C#4', 'F#2', 'E-4', '6.11', 'E4', '10.2', 'E4', '6.11', '6.11', 'C#4', 'F#2', 'C#4', '6.11', '10.2', 'C#4', 'F#2', 'E-4', '6.11', 'E4', '10.2', 'E4', '6.11', '6.11', 'C#4', 'F#2', 'C#4', '6.11', '10.2', 'C#4', 'F#2', 'E-4', '6.11', 'E4', '10.2', 'E4', '6.11', '6.11', 'C#4', 'F#2', 'C#4', '6.11', '10.2', 'C#4', 'F#2', 'E-4', '6.11', 'E4', '10.2', 'E4', '6.11', '6.11', 'C#4', 'F#2', 'C#4', '6.11', '10.2', 'C#4', 'F#2', 'E-4', '6.11', 'E4', '10.2', 'E4', '6.11', '6.11', 'C#4', 'F#2', 'C#4', '6.

In [2]:
import numpy
from keras.utils import np_utils

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)



network_input, network_output = prepare_sequences(notes, n_vocab)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(128))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model



model = create_network(network_input, n_vocab)

In [4]:
from keras.callbacks import ModelCheckpoint
import h5py

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=4, batch_size=6, callbacks=callbacks_list)


    
train(model, network_input, network_output)

Epoch 1/4
1955/1955 [==============================] - 398s 203ms/step - loss: 4.0359
Epoch 2/4
1955/1955 [==============================] - 349s 179ms/step - loss: 3.8645
Epoch 3/4
1955/1955 [==============================] - 323s 165ms/step - loss: 3.8309
Epoch 4/4
1955/1955 [==============================] - 332s 170ms/step - loss: 3.8434


In [5]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

NameError: name 'pitchnames' is not defined